![](https://media.giphy.com/media/tWRJEZeQeHAqs/giphy.gif)

<body>
    <font size=6 color="red">Background checkup</font>
</body>
 

The basic idea of analyzing the Zomato dataset is to get a fair idea about the factors affecting the establishment of different types of restaurant at different places in Bengaluru, aggregate rating of each restaurant, Bengaluru being one such city has more than 12,000 restaurants with restaurants serving dishes from all over the world. With each day new restaurants opening the industry has’nt been saturated yet and the demand is increasing day by day. Inspite of increasing demand it however has become difficult for new restaurants to compete with established restaurants. Most of them serving the same food. Bengaluru being an IT capital of India. Most of the people here are dependent mainly on the restaurant food as they don’t have time to cook for themselves. With such an overwhelming demand of restaurants it has therefore become important to study the demography of a location.

- What kind of a food is more popular in a locality. 
- Do the entire locality loves vegetarian food. If yes then is that locality populated by a particular sect of people for eg. Jain, Marwaris, Gujaratis who are mostly vegetarian. These kind of analysis can be done using the data, by studying the factors such as • Location of the restaurant • Approx Price of food  Theme based restaurant or not 
-  Which locality of that city serves that cuisines with maximum number of restaurants • The needs of people who are striving to get the best cuisine of the neighborhood • Is a particular neighborhood famous for its own kind of food.

<font size=5 color="violet">If you like my work,please consider <font color='red'>upvoting</font> my kernel.</font>

<font size=5 color='blue'> What is in this kernel</font>

1. [Loading required libraries](#1)
2. [Getting basic ideas about the data](#2)
3. [Exploratory data analysis](#3)

     1. [ Which are the top restaurant chains in Bangaluru?](#4)
     2. [How many of the restuarants do not accept online orders?](#5)
     3. [What is the ratio b/w restaurants that provide and do not provide table booking ?](#25)
     4. [ Rating distribution](#6)
     5. [Is there any difference b/w votes of restaurants accepting and not accepting online orders?](#7)
     6. [Which are the most common restaurant type in Banglore?](#8)
     7. [Cost factor ?](#9)
     8. [Which are the foodie areas?](#10)
     9. [Which are the most common cuisines in each locations?](#11)
     10. [Extracting location information using Geopy](#12)
     11. [Heatmap of restaurant count on each location](13)
     12. [Which are  the most popular cuisines of Bangalore?](#14)
     13. [Heatmap of North Indian restaurants](#15)
     14. [What about South  Indian cuisines?](#16)
     15. [Analysing Restaurant chains ](#17)
            16. [Which are the most popular casual dining restaurant chains?](#18)
                   1. [Where are their outlets located?](#19)
            17. [Top quick bites restaurant chains in Banglore](#20)
                   1. [Where are their outlets located?](#21)
            18. [Top Cafes of Banglore?](#22)
                   1. [Where are their outlets located?](#23)
            19. [Wordcloud of dishes liked by cuisines.](#24)
     20. [Analysing Reviews](#26)
            21. [Wordcloud of restaurant reviews](#27)
            22. [Rating distribution](#28)
4. [Topic modelling](#33)
     23. [Topic modelling for positive comments](#34)
     24. [Topic modelling for positive comments](#35)
         
            
5. [Sentimental Analysis on Reviews](#29)
     23. [Data preparation](#30)
     24. [Building our model](#31)
     25. [Validation](#32)
     

## [Loading libraries]()<a id="1"></a> <br>


In [ ]:
import numpy as np 
import pandas as pd
import os
import seaborn as sns
print(os.listdir("../input"))
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from wordcloud import WordCloud
from geopy.geocoders import Nominatim
from folium.plugins import HeatMap
import folium
from tqdm import tqdm
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.colors as mcolors
from sklearn.manifold import TSNE
from gensim.models import word2vec
import nltk


In [ ]:
from matplotlib import rcParams

In [ ]:
df=pd.read_csv("../input/zomato.csv")

## Getting Basic Ideas


In [ ]:
#print("dataset contains {} rows and {} columns".format(df.shape[0],df.shape[1]))
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

**Columns description**

- **url**
contains the url of the restaurant in the zomato website

- **address**
contains the address of the restaurant in Bengaluru

- **name**
contains the name of the restaurant

- **online_order**
whether online ordering is available in the restaurant or not

- **book_table**
table book option available or not

- **rate**
contains the overall rating of the restaurant out of 5

- **votes**
contains total number of rating for the restaurant as of the above mentioned date

- **phone**
contains the phone number of the restaurant

- **location**
contains the neighborhood in which the restaurant is located

- **rest_type**
restaurant type

- **dish_liked**
dishes people liked in the restaurant

- **cuisines**
food styles, separated by comma

- **approx_cost(for two people)**
contains the approximate cost for meal for two people

- **reviews_list**
list of tuples containing reviews for the restaurant, each tuple 

- **menu_item**
contains list of menus available in the restaurant

- **listed_in(type)**
type of meal

- **listed_in(city)**
contains the neighborhood in which the restaurant is listed


## Exploratory data analysis

In [ ]:
zomato_orginal_copy=df.copy()

In [ ]:
zomato_orginal_copy.rename(columns={'approx_cost(for two people)': 'average_cost', 'listed_in(city)': 'locality','listed_in(type)': 'meal_type'}, inplace=True)

In [ ]:
zomato_orginal_copy.columns

## Which are the top restaurant chains in Bangaluru?


In [ ]:
no_of_the_restaurants=zomato_orginal_copy['name'].value_counts()

In [ ]:
no_of_the_restaurants

In [ ]:
restaurant_name = list(zomato_orginal_copy['name'].astype(str).values)
restaurant_name_word_list = []
for lista in [c.split(',') for c in restaurant_name]:
    for word in lista:
        restaurant_name_word_list.append(word.strip())
        
# Creating a Counter for unique options and generating the wordcloud
restaurant_name_wc_dict = Counter(restaurant_name_word_list)

wordcloud = WordCloud(width=1280, height=820, collocations=False, random_state=42, 
                      colormap='CMRmap', background_color='white').generate_from_frequencies(restaurant_name_wc_dict)
# Visualizing the WC created and the total for each cuisine
fig, axs = plt.subplots(1, 2, figsize=(20, 12))
ax1 = axs[0]
ax2 = axs[1]
ax1.imshow(wordcloud)
ax1.axis('off')
ax1.set_title('WordCloud for most famous restaurants chains in Bangaluru', size=18, pad=20)
sns.barplot(x=no_of_the_restaurants[:20],y=no_of_the_restaurants.index[:20],palette="CMRmap",ax=ax2)
#sns.barplot(x=chains,y=chains.index,palette='deep')
ax2.set_title("Most famous restaurants chains in Bangaluru",size=20)
plt.show()

In [ ]:
#rcParams['figure.figsize'] = 15,7
#sns.barplot(x=no_of_the_restaurants[:20],y=no_of_the_restaurants.index[:20],palette="CMRmap")
#sns.barplot(x=chains,y=chains.index,palette='deep')
#plt.title("Most famous restaurants chains in Bangaluru")
#plt.ylabel("Number of outlets")

- As you can see **Cafe coffee day,Onesta,Just Bake** has the most number of outlets in and around bangalore.
- This is rather interesting,we will inspect each of them later.

In [ ]:
Locality=zomato_orginal_copy['locality'].value_counts()
Locality

## Most Famous Foodie Areas In Bangalore

In [ ]:
rcParams['figure.figsize'] = 15,7
g = sns.countplot(x="locality",data=zomato_orginal_copy, palette = "Set1")
g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right")
g 
plt.title('Most Famous Foodie Areas In Bangalore',size = 20)

## How many of the restuarants accept & don't accept online orders?


In [ ]:
type_of_order=zomato_orginal_copy['online_order'].value_counts()
type_of_order

In [ ]:
colors = ['#FF0099', '#FFFF33']
colors_1 = ['#9900CC', '#00CCFF']

from plotly.subplots import make_subplots
fig= make_subplots(rows=1, cols=2,shared_xaxes=False,specs=[[{"type": "pie"}, {"type": "bar"}]])

fig.add_trace(go.Pie(labels=type_of_order.index,values=type_of_order.values,textinfo='percent+label',name='',
            marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2))),row=1,col=1)
fig.add_trace(go.Bar(x=type_of_order.index,y=type_of_order.values,name='',
                         marker=dict(color=colors_1,line=dict(color='#000000', width=2))),
                          row=1,col=2)
fig.update_layout(title="Accepting vs not accepting online orders",xaxis_title="category",yaxis_title="count" ,width=1000,height=500,showlegend=True)
fig.show()

1. As clearly indicated,almost 60 per cent of restaurants in Banglore accepts online orders.
2. Nearly 40 per cent of the restaurants do not accept online orders.
3. This might be because of the fact that these restaurants cannot afford to pay commission to zomoto for giving them orders online.
   zomato may want to consider giving them some more benefits if they want to increse the number of restaurants serving their customers online.


In [ ]:
#x=df['online_order'].value_counts()
#colors = ['#FEBFB3', '#E1396C']

#trace=go.Pie(labels=x.index,values=x,textinfo="value",
            #marker=dict(colors=colors, 
            #               line=dict(color='#000000', width=2)))
#layout=go.Layout(title="Accepting vs not accepting online orders",width=500,height=500)
#fig=go.Figure(data=[trace],layout=layout)
#py.iplot(fig, filename='pie_chart_subplots')
    
    
    

## What is the ratio b/w restaurants that provide and do not provide table booking facility ?



In [ ]:
table_booking=zomato_orginal_copy['book_table'].value_counts()
table_booking

In [ ]:
colors = ['#CC0099', '#FF66FF']
colors_1 = ['#FF6666', '#FFFF33']

fig= make_subplots(rows=1, cols=2,shared_xaxes=False,specs=[[{"type": "pie"}, {"type": "bar"}]])

fig.add_trace(go.Pie(labels=table_booking.index,values=table_booking.values,textinfo='percent+label',name='',
            marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2))),row=1,col=1)
fig.add_trace(go.Bar(x=table_booking.index,y=table_booking.values,name='',
                         marker=dict(color=colors_1,line=dict(color='#000000', width=2))),
                          row=1,col=2)
fig.update_layout(title="Offering vs not offering table booking facility ",xaxis_title="category",yaxis_title="count" ,width=1000,height=500,showlegend=True)
fig.show()

1. Almost 90 percent of restaurants in Banglore do not provide table booking facility.
2. In India you cannot find table booking facility in any average restaurants,usually only five star restaurants provides table booking.
3. We will inspect this further.

In [ ]:
#x=df['book_table'].value_counts()
#colors = ['#96D38C', '#D0F9B1']

#trace=go.Pie(labels=x.index,values=x,textinfo="value",
            #marker=dict(colors=colors, 
                           #line=dict(color='#000000', width=2)))
#layout=go.Layout(title="Table booking",width=500,height=500)
#fig=go.Figure(data=[trace],layout=layout)
#py.iplot(fig, filename='pie_chart_subplots')
    

In [ ]:
zomato_orginal_copy.average_cost

In [ ]:
zomato_orginal_copy['average_cost']=zomato_orginal_copy['average_cost'].astype(str)
zomato_orginal_copy['average_cost']=zomato_orginal_copy['average_cost'].apply(lambda y:y.replace(',',''))
zomato_orginal_copy['average_cost']=zomato_orginal_copy['average_cost'].apply(lambda y:float(y))

In [ ]:
zomato_orginal_copy['average_cost'].isnull().mean()

In [ ]:
zomato_orginal_copy['average_cost'].dropna()

In [ ]:
zomato_orginal_copy['average_cost'].isnull().mean()

## [Rating distribution]()<a id="6"></a> <br>


In [ ]:
zomato_rate=zomato_orginal_copy['rate'].copy()

In [ ]:
rating=zomato_rate.dropna().apply(lambda x : float(x.split('/')[0]) if (len(x)>3)  else np.nan ).dropna()

In [ ]:
sns.distplot(rating,bins=20)

In [ ]:

#plt.figure(figsize=(6,5))
#rating=df['rate'].dropna().apply(lambda x : float(x.split('/')[0]) if (len(x)>3)  else np.nan ).dropna()
#sns.distplot(rating,bins=20)

1. Almost more than 50 percent of restaurants has rating between 3 and 4.
2. Restaurants having rating more than 4.5 are very rare.

In [ ]:
zomato_orginal_copy['average_cost']

In [ ]:

cost_dist=zomato_orginal_copy[['rate','average_cost','online_order','book_table','votes','cuisines','name','rest_type','location','meal_type']].dropna()
cost_dist['rate']=cost_dist['rate'].apply(lambda x: float(x.split('/')[0]) if len(x)>3 else 0)
#cost_dist['average_cost']=cost_dist['average_cost'].apply(lambda x: int(x.replace(',','')))


## [Cost vs rating]()

In [ ]:
plt.figure(figsize=(10,7))
sns.scatterplot(x="rate",y='average_cost',hue='online_order',data=cost_dist)
plt.show()

In [ ]:
import plotly.express as px
fig = px.scatter(cost_dist, x="rate", y="average_cost", color="online_order", hover_data=['votes'])
fig.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.scatterplot(x="rate",y='average_cost',hue='book_table',data=cost_dist)
plt.show()

In [ ]:
fig = px.scatter(cost_dist, x="rate", y="average_cost", color="book_table", hover_data=['votes'])
fig.show()

## Cost Vs Voting

In [ ]:
plt.figure(figsize=(10,7))
sns.scatterplot(x="votes",y='average_cost',hue='online_order',data=cost_dist)
plt.show()

### [Distribution of cost for two people]()

## Cost Factor

In [ ]:
plt.figure(figsize=(6,6))
sns.distplot(cost_dist['average_cost'])
plt.show()

- We can see that the distribution if left skewed.
- This means almost 90percent of restaurants serve food for budget less than 1000 INR.($15)

In [ ]:
cost_accepting_online_order=cost_dist[cost_dist['online_order']=="Yes"]['average_cost']
trace0=go.Box(y=cost_accepting_online_order,name="accepting online orders",
              marker = dict(
        color = 'midnightblue',
    ))

data=[trace0]
layout=go.Layout(title="Box plot of paying cost for restaurants which provide online order facility",width=800,height=500,yaxis=dict(title="Price"))
fig=go.Figure(data=data,layout=layout)
fig.show()

## Is there any difference b/w votes of restaurants accepting and not accepting online orders?


In [ ]:
votes_yes=zomato_orginal_copy[zomato_orginal_copy['online_order']=="Yes"]['votes']
trace0=go.Box(y=votes_yes,name="accepting online orders",
              marker = dict(
        color = 'rgb(214, 12, 140)',
    ))

votes_no=zomato_orginal_copy[zomato_orginal_copy['online_order']=="No"]['votes']
trace1=go.Box(y=votes_no,name="Not accepting online orders",marker = dict(
        color = 'rgb(10, 120, 120)',
    ))

layout = go.Layout(
    title = "Box Plots of votes",width=800,height=500
)

data=[trace0,trace1]
fig=go.Figure(data=data,layout=layout)
fig.show()

1. Yes,you can observe that median number of votes for both categories vary.
2. Restaurants accepting online orders tend to get more votes from customers as there is a rating option poping up after each order through zomato application.

In [ ]:
cost_dist['rate']

## Is there any difference b/w rating of restaurants accepting and not accepting online orders?

In [ ]:
rate_yes=cost_dist[cost_dist['online_order']=="Yes"]['rate']
trace0=go.Box(y=rate_yes,name="accepting online orders",
              marker = dict(
        color = 'peru',
    ))

rate_no=cost_dist[cost_dist['online_order']=="No"]['rate']
trace1=go.Box(y=rate_no,name="Not accepting online orders",marker = dict(
        color = 'royalblue',
    ))

layout = go.Layout(
    title = "Box Plots of rating",width=800,height=500
)

data=[trace0,trace1]
fig=go.Figure(data=data,layout=layout)
fig.show()

In [ ]:
from matplotlib.gridspec import GridSpec

## How the offering of Online Order service impact on Rate and Approx Cost?

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(15, 12))

# Axis definition with GridSpec
gs = GridSpec(2, 5, figure=fig)
ax2 = fig.add_subplot(gs[0, :3])
ax3 = fig.add_subplot(gs[0, 3:])
ax4 = fig.add_subplot(gs[1, :3])
ax5 = fig.add_subplot(gs[1, 3:])

# First Line (01) - Rate
sns.kdeplot(cost_dist.query('rate > 0 & online_order == "Yes"')['rate'], ax=ax2,
             color='darkslateblue', shade=True, label='With Online Order Service')
sns.kdeplot(cost_dist.query('rate > 0 & online_order == "No"')['rate'], ax=ax2,
             color='lightsalmon', shade=True, label='Without Online Order Service')
ax2.set_title('Restaurants Rate Distribution by Online Order Service Offer', color='dimgrey', size=14)
sns.boxplot(x='online_order', y='rate', data=cost_dist, palette=['darkslateblue', 'lightsalmon'], ax=ax3)
ax3.set_title('Box Plot for Rate and Online Order Service', color='dimgrey', size=14)
# First Line (01) - Cost
sns.kdeplot(cost_dist.query('average_cost > 0 & online_order == "Yes"')['average_cost'], ax=ax4,
             color='darkslateblue', shade=True, label='With Online Order Service')
sns.kdeplot(cost_dist.query('average_cost > 0 & online_order == "No"')['average_cost'], ax=ax4,
             color='lightsalmon', shade=True, label='Without Online Order Service')
ax4.set_title('Restaurants Approx Cost Distribution by Online Order Service Offer', color='dimgrey', size=14)
sns.boxplot(x='online_order', y='average_cost', data=cost_dist, palette=['darkslateblue', 'lightsalmon'], ax=ax5)
ax5.set_title('Box Plot for Cost and Online Order Service', color='dimgrey', size=14)


# Customizing plots
#for ax in [ax2, ax3, ax4, ax5]:
    #format_spines(ax, right_border=False)
plt.tight_layout()

Now if we look for online order service, it's fair to say that the customer don't give importance for this service as much they give for book table one. The rate distribution shows customers usually give highest rates for restaurants who offer online order service (even if the difference is little).

Looking at the cost comparison between restaurants who offer online order service, we can conclude that they are almost the same.


## How the offering of Book Table service impact on Rate and Approx Cost?

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(15, 12))

# Axis definition with GridSpec
gs = GridSpec(2, 5, figure=fig)
ax2 = fig.add_subplot(gs[0, :3])
ax3 = fig.add_subplot(gs[0, 3:])
ax4 = fig.add_subplot(gs[1, :3])
ax5 = fig.add_subplot(gs[1, 3:])

# First Line (01) - Rate
sns.kdeplot(cost_dist.query('rate > 0 & book_table == "Yes"')['rate'], ax=ax2,
             color='mediumseagreen', shade=True, label='With Book Table Service')
sns.kdeplot(cost_dist.query('rate > 0 & book_table == "No"')['rate'], ax=ax2,
             color='crimson', shade=True, label='Without Book Table Service')
ax2.set_title('Restaurants Rate Distribution by Book Table Service Offer', color='dimgrey', size=14)
sns.boxplot(x='book_table', y='rate', data=cost_dist, palette=['mediumseagreen', 'crimson'], ax=ax3)
ax3.set_title('Box Plot for Rate and Book Table Service', color='dimgrey', size=14)
# First Line (01) - Cost
sns.kdeplot(cost_dist.query('average_cost > 0 & book_table == "Yes"')['average_cost'], ax=ax4,
             color='mediumseagreen', shade=True, label='With Book Table Service')
sns.kdeplot(cost_dist.query('average_cost > 0 & book_table == "No"')['average_cost'], ax=ax4,
             color='crimson', shade=True, label='Without Book Table Service')
ax4.set_title('Restaurants Approx Cost Distribution by Book Table Service Offer', color='dimgrey', size=14)
sns.boxplot(x='book_table', y='average_cost', data=cost_dist, palette=['mediumseagreen', 'crimson'], ax=ax5)
ax5.set_title('Box Plot for Cost and Book Table Service', color='dimgrey', size=14)


# Customizing plots
#for ax in [ax2, ax3, ax4, ax5]:
    #format_spines(ax, right_border=False)
plt.tight_layout()

The distplot and the boxplot above show us the importance given by the customers for book table service in restaurants: on the first line we notice that, in general, restaurants with book table service usually receive highest rate notes. Meanwhile, we can conclude by the second line that those same restaurants are usually more expensive.



## Is there any difference b/w rating of restaurants offering and not offering  table booking facilities?

In [ ]:
rate_yes_book_table=cost_dist[cost_dist['book_table']=="Yes"]['rate']
trace0=go.Box(y=rate_yes_book_table,name="Offering table booking facility",
              marker = dict(
        color = 'red',
    ))

rate_no_book_table=cost_dist[cost_dist['book_table']=="No"]['rate']
trace1=go.Box(y=rate_no_book_table,name="Not Offering table booking facility",marker = dict(
        color ='limegreen',
    ))

layout = go.Layout(
    title = "Box Plots of rating",width=800,height=500
)

data=[trace0,trace1]
fig=go.Figure(data=data,layout=layout)
fig.show()

In [ ]:
cost_offering_table_booking_facility=cost_dist[cost_dist['book_table']=="Yes"]['average_cost']
trace1=go.Box(y=cost_offering_table_booking_facility,name="providing booking table facility",
              marker = dict(
        color = 'orangered',
    ))
data=[trace1]
layout=go.Layout(title="Box plot of paying cost for restaurants which provide table booking facility",width=800,height=500,yaxis=dict(title="Price"))
fig=go.Figure(data=data,layout=layout)
fig.show()

## Is there any difference b/w votes of restaurants offering and not offering  table booking facilities?

In [ ]:
votes_yes_table_book=zomato_orginal_copy[zomato_orginal_copy['book_table']=="Yes"]['votes']
trace0=go.Box(y=votes_yes_table_book,name="Offering table booking facility",
              marker = dict(
        color = 'orange',
    ))

votes_no_table_book=zomato_orginal_copy[zomato_orginal_copy['book_table']=="No"]['votes']
trace1=go.Box(y=votes_no_table_book,name="Not Offering table booking facility",marker = dict(
        color = 'purple',
    ))

layout = go.Layout(
    title = "Box Plots of votes",width=800,height=500
)

data=[trace0,trace1]
fig=go.Figure(data=data,layout=layout)
fig.show()

1. The median approximate cost for ordering in online of two people is 400 for a single meal.
2. 50 percent of restaurants charge for online ordering is  between 300 and 450 for single meal for two people.


In [ ]:


'''votes_yes=df[df['online_order']=="Yes"]['votes']
trace0=go.Box(y=votes_yes,name="accepting online orders",
              marker = dict(
        color = 'rgb(214, 12, 140)',
    ))

votes_no=df[df['online_order']=="No"]['votes']
trace1=go.Box(y=votes_no,name="Not accepting online orders",
              marker = dict(
        color = 'rgb(0, 128, 128)',
    ))

layout = go.Layout(
    title = "Box Plots of votes",width=800,height=500
)

data=[trace0,trace1]
fig=go.Figure(data=data,layout=layout)
py.iplot(fig)'''




## Which are the most common type of meals in Banglore?

In [ ]:
types_of_meal=zomato_orginal_copy['meal_type'].value_counts()
types_of_meal

In [ ]:
sns.barplot(x=types_of_meal.index,y=types_of_meal.values,palette='Set1')
plt.title("High Demand Restaurant Types")
plt.ylabel("count")
plt.xlabel("Different types Of Meals")

In [ ]:
cost_dist.columns

## How the rate and approx cost are distributed among restaurant types?

In [ ]:
rest_types = list(cost_dist['meal_type'].value_counts().index)
colors = ['darkslateblue', 'mediumseagreen', 'gray', 'salmon', 'cornflowerblue', 'cadetblue', 'gold']

fig, axs = plt.subplots(2, 1, figsize=(17, 15))
for r_type in rest_types:
    idx = rest_types.index(r_type)
    kde_data = cost_dist[(cost_dist['rate']>0) & (cost_dist['meal_type'] == r_type)]
    sns.kdeplot(kde_data['rate'], ax=axs[0], color=colors[idx], shade=True, label=r_type)
    sns.kdeplot(kde_data['average_cost'], ax=axs[1], color=colors[idx], shade=True, label=r_type)
# Customizing charts
axs[0].set_title('Distribution of Rate by Different Types Of Meal', color='dimgrey', size=18)
axs[1].set_title('Distribution of Approx Cost by Different Types Of Meal', color='dimgrey', size=18)
#for ax in axs:
    #format_spines(ax, right_border=False)
plt.tight_layout()

## Which are the most common restaurant type in Banglore?


In [ ]:

restaurant_type=zomato_orginal_copy['rest_type'].value_counts()
restaurant_type

In [ ]:
rest_type = list(cost_dist['rest_type'].astype(str).values)
rest_type_word_list = []
for lista in [c.split(',') for c in rest_type]:
    for word in lista:
        rest_type_word_list.append(word.strip())
        
# Creating a Counter for unique options and generating the wordcloud
rest_type_wc_dict = Counter(rest_type_word_list)

wordcloud = WordCloud(width=1280, height=820, collocations=False, random_state=42, 
                      colormap='gist_ncar', background_color='white').generate_from_frequencies(rest_type_wc_dict)
# Visualizing the WC created and the total for each cuisine
fig, axs = plt.subplots(1, 2, figsize=(20, 12))
ax1 = axs[0]
ax2 = axs[1]
ax1.imshow(wordcloud)
ax1.axis('off')
ax1.set_title('WordCloud for different restaurant types available', size=18, pad=20)
sns.barplot(restaurant_type[:20],restaurant_type.index[:20],palette='gist_ncar',ax=ax2)
ax2.set_title("High Demand Restaurant Types",size=20)
#plt.xlabel("count")
#plt.ylabel("Different types Of Restaurant")
plt.show()

1. No doubt about this as Banglore is known as the tech capital of India,people having busy and modern life will prefer Quick Bites.
2. We can observe tha Quick Bites type restaurants dominates.

In [ ]:
rest_types1 = list(cost_dist['rest_type'].value_counts().index[:10])
colors = ['royalblue', 'mediumseagreen', 'magenta', 'salmon', 'cornflowerblue', 'darkcyan', 'gold','red','orchid','olive']

fig, axs = plt.subplots(2, 1, figsize=(17, 15))
for r_type in rest_types1:
    idx = rest_types1.index(r_type)
    kde_data = cost_dist[(cost_dist['rate']>0) & (cost_dist['rest_type'] == r_type)]
    sns.kdeplot(kde_data['rate'], ax=axs[0], color=colors[idx], shade=True, label=r_type)
    sns.kdeplot(kde_data['average_cost'], ax=axs[1], color=colors[idx], shade=True, label=r_type)
# Customizing charts
axs[0].set_title('Distribution of Rate by Different Types Of Restaurant', color='dimgrey', size=18)
axs[1].set_title('Distribution of Approx Cost by Different Types Of Restaurant', color='dimgrey', size=18)
#for ax in axs:
    #format_spines(ax, right_border=False)
plt.tight_layout()

In [ ]:
#plt.figure(figsize=(7,7))
#rest=df['rest_type'].value_counts()[:20]
#sns.barplot(restaurant_type[:20],restaurant_type.index[:20],palette='gist_ncar')
#plt.title("High Demand Restaurant Types")
#plt.xlabel("count")
#plt.ylabel("Different types Of Restaurant")

### Finding Best budget Restaurants in any location

- I have implemented a simple filtering mechanism to find best budget restaurants in any locations in Bangalore.
- You can pass location and restaurant type as parameteres,function will return name of restaurants.


In [ ]:
#cost_dist=df[['rate','approx_cost(for two people)','location','name','rest_type']].dropna()
#cost_dist['rate']=cost_dist['rate'].apply(lambda x: float(x.split('/')[0]) if len(x)>3 else 0)
#cost_dist['approx_cost(for two people)']=cost_dist['approx_cost(for two people)'].apply(lambda x: int(x.replace(',','')))

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
def return_budget(location,rest):
    budget=cost_dist[(cost_dist['average_cost']<=600) & (cost_dist['location']==location) & 
                     (cost_dist['rate']>4) & (cost_dist['rest_type']==rest)]
    names=budget['name'].value_counts()
    #sns.barplot(restaurant_type[:20],restaurant_type.index[:20],palette='gist_ncar')
    #plt.title("High Demand Restaurant Types")
    #plt.xlabel("count")
    #plt.ylabel("Different types Of Restaurant")
    trace0=go.Bar(x=names.index,y=names.values,
                         marker=dict(color='darkcyan',line=dict(color='#000000', width=2)))
    layout=go.Layout(title="Best Budget Restaurant ",xaxis_title="name of the outlets",yaxis_title="count" ,width=1000,height=500,showlegend=True)
    data=[trace0]
    fig=go.Figure(data=data,layout=layout)
    #fig.show()
    
    Top_cuisines=budget['cuisines'].value_counts()
    trace1=go.Bar(x=Top_cuisines.index,y=Top_cuisines.values,
                         marker=dict(color='steelblue',line=dict(color='#000000', width=2)))
    layout1=go.Layout(xaxis_title="name of the cuisines",yaxis_title="count" ,width=1000,height=500,showlegend=True)
    data1=[trace1]
    fig2=go.Figure(data=data1,layout=layout1)
    
    
    colors = ['#FF0099', '#FFFF33']
    colors_1 = ['#9900CC', '#00CCFF']
    
    type_of_order=budget['online_order'].value_counts()
    fig1= make_subplots(rows=1, cols=2,shared_xaxes=False,specs=[[{"type": "pie"}, {"type": "bar"}]])

    fig1.add_trace(go.Pie(labels=type_of_order.index,values=type_of_order.values,textinfo='percent+label',name='',
            marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2))),row=1,col=1)
    fig1.add_trace(go.Bar(x=type_of_order.index,y=type_of_order.values,name='',
                         marker=dict(color=colors_1,line=dict(color='#000000', width=2))),
                          row=1,col=2)
    fig1.update_layout(title="Accepting vs not accepting online orders",xaxis_title="category",yaxis_title="count" ,width=1000,height=500,showlegend=True)
    c=fig.show()
    d=fig1.show()
    e=fig2.show()
    return(c,d,e)

In [ ]:
return_budget('BTM',"Quick Bites")

## Which are the foodie areas?


In [ ]:
Locations_Of_The_Restaurant=zomato_orginal_copy['location'].value_counts()
Locations_Of_The_Restaurant

In [ ]:
Location_restaurant = list(zomato_orginal_copy['location'].astype(str).values)
Location_restaurant_type_word_list = []
for lista in [c.split(',') for c in Location_restaurant]:
    for word in lista:
        Location_restaurant_type_word_list.append(word.strip())
        
# Creating a Counter for unique options and generating the wordcloud
Location_restaurant_type_wc_dict = Counter(Location_restaurant_type_word_list)

wordcloud = WordCloud(width=1280, height=820, collocations=False, random_state=42, 
                       background_color='white').generate_from_frequencies(Location_restaurant_type_wc_dict)
# Visualizing the WC created and the total for each cuisine
fig, axs = plt.subplots(1, 2, figsize=(20, 12))
ax1 = axs[0]
ax2 = axs[1]
ax1.imshow(wordcloud)
ax1.axis('off')
ax1.set_title('WordCloud For Most famous Foodie Areas in Bangaluru', size=18, pad=20)
sns.barplot(x=Locations_Of_The_Restaurant[:20],y=Locations_Of_The_Restaurant.index[:20],ax=ax2)
ax2.set_title("Most famous Foodie Areas in Bangaluru",size=20)
#plt.xlabel("count")
#plt.ylabel("Different types Of Restaurant")
plt.show()

1. We can see that **BTM,HSR and Koranmangala 5th block** has the most number of restaurants.
2. BTM dominates the section by having more than 5000 restaurants.

In [ ]:
#rcParams['figure.figsize'] = 15,7
#sns.barplot(x=Locations_Of_The_Restaurant[:20],y=Locations_Of_The_Restaurant.index[:20])
#plt.title("Most famous Foodie Areas in Bangaluru")
#plt.ylabel("Top Foodie Areas")

## Which are the most common cuisines in each locations?


In [ ]:
df_2=zomato_orginal_copy.groupby(['location','cuisines']).agg('count')
data=df_2.sort_values(['url'],ascending=False).groupby(['location'],
                as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False).head(3))['url'].reset_index().rename(columns={'url':'count'})


In [ ]:

data.head(10)

## Extracting location information using Geopy

In [ ]:
locations=pd.DataFrame({"Name":zomato_orginal_copy['location'].unique()})
locations['Name']=locations['Name'].apply(lambda x: "Bangalore " + str(x))
lat_lon=[]
geolocator=Nominatim(user_agent="app")
for location in locations['Name']:
    location = geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:    
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)


locations['geo_loc']=lat_lon
locations.to_csv('locations.csv',index=False)

In [ ]:
#locations["Name"]=locations['Name'].apply(lambda x :  x.replace("Bangalore","")[1:])
locations.head()

- We have found out latitude and longitude of each location listed in the dataset using geopy.
- This is used to plot maps.

## Heatmap of restaurant count on each location

In [ ]:
zomato_orginal_copy['location']

In [ ]:
Rest_locations=pd.DataFrame(zomato_orginal_copy['location'].value_counts().reset_index())

In [ ]:
Rest_locations

In [ ]:
locations=pd.DataFrame({"Name":df['location'].unique()})
locations['Name']=locations['Name'].apply(lambda x: "Bangalore " + str(x))
lat_lon=[]
geolocator=Nominatim(user_agent="app")
for location in locations['Name']:
    location = geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:    
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)
locations['geo_loc']=lat_lon
locations.to_csv('locations.csv',index=False)

In [ ]:

locations['geo_loc']=lat_lon
locations.to_csv('locations.csv',index=False)

In [ ]:
locations["Name"]=locations['Name'].apply(lambda x :  x.replace("Bangalore","")[1:])

In [ ]:
locations.head()

In [ ]:
Rest_locations=pd.DataFrame(df['location'].value_counts().reset_index())
Rest_locations.columns=['Name','count']
Rest_locations=Rest_locations.merge(locations,on='Name',how="left").dropna()
Rest_locations['count'].max()

In [ ]:
'''Rest_locations=pd.DataFrame(zomato_orginal_copy['location'].value_counts().reset_index())
Rest_locations.columns=['Name','count']
Rest_locations=Rest_locations.merge(locations,on='Name',how="left").dropna()
Rest_locations['count'].max()'''

In [ ]:
Rest_locations

In [ ]:
def generateBaseMap(default_location=[12.97, 77.59], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [ ]:
lat,lon=zip(*np.array(Rest_locations['geo_loc']))
Rest_locations['lat']=lat
Rest_locations['lon']=lon
basemap=generateBaseMap()
HeatMap(Rest_locations[['lat','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)

In [ ]:
basemap

1. It is clear that restaurants tend to concentrate in central bangalore area.
2. The clutter of restaurants lowers are we move away from central.
3. So,potential restaurant entrepreneurs can refer this and find out good locations for their venture.

## Which are  the most popular cuisines of Bangalore?

In [ ]:
Popular_Cuisines=zomato_orginal_copy['cuisines'].value_counts()
#sns.barplot(x=Popular_Cuisines[:20],y=Popular_Cuisines.index[:20],palette='nipy_spectral')
#plt.title("Most famous Cuisines in Bangaluru")
#plt.ylabel("Top Cuisines")

In [ ]:
cuisines = list(zomato_orginal_copy['cuisines'].astype(str).values)
cuisines_type_word_list = []
for lista in [c.split(',') for c in cuisines]:
    for word in lista:
        cuisines_type_word_list.append(word.strip())
        
# Creating a Counter for unique options and generating the wordcloud
cuisines_type_wc_dict = Counter(cuisines_type_word_list)

wordcloud = WordCloud(width=1280, height=820, collocations=False, random_state=42, 
                       background_color='white').generate_from_frequencies(cuisines_type_wc_dict)
# Visualizing the WC created and the total for each cuisine
fig, axs = plt.subplots(1, 2, figsize=(20, 12))
ax1 = axs[0]
ax2 = axs[1]
ax1.imshow(wordcloud)
ax1.axis('off')
ax1.set_title('WordCloud For Most famous Cuisines in Bangaluru', size=18, pad=20)
sns.barplot(x=Popular_Cuisines[:20],y=Popular_Cuisines.index[:20],palette='nipy_spectral',ax=ax2)
ax2.set_title("Most famous Cuisines in Bangaluru",size=20)
#plt.xlabel("count")
#plt.ylabel("Different types Of Restaurant")
plt.show()

1. We can observe that **North Indian,chinese,South Indian and Biriyani** are most common.
2. Is this imply the fact that Banglore is more influenced by North Indian culture more than South?
- We will inspect further......

## Heatmap of North Indian restaurants

In [ ]:
def produce_data(col,name):
    data= pd.DataFrame(zomato_orginal_copy[zomato_orginal_copy[col]==name].groupby(['location'],as_index=False)['url'].agg('count'))
    data.columns=['Name','count']
    print(data.head())
    data=data.merge(locations,on="Name",how='left').dropna()
    data['lan'],data['lon']=zip(*data['geo_loc'].values)
    return data.drop(['geo_loc'],axis=1)



In [ ]:
North_India=produce_data('cuisines','North Indian')
North_India

In [ ]:
basemap=generateBaseMap()
HeatMap(North_India[['lan','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)
basemap

1. Interesting to see a cluster of North Indian Restaurants in South Bangalore area !
2. This may indicate that these areas are more populated by North Indians.

## What about South  Indian cuisines?

In [ ]:
food=produce_data('cuisines','South Indian')
basemap=generateBaseMap()
HeatMap(food[['lan','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)
basemap

1. They tend to span all over Bangalore.
2. South Indian cuisines tend to cluster near central Bangalore.

## Analysing Restaurant chains

In [ ]:
df_11=df.groupby(['rest_type','name']).agg('count')
#datas=df_11.sort_values(['url'],ascending=False).groupby(['rest_type'],
                #as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False).head(3))['url'].reset_index().rename(columns={'url':'count'})


In [ ]:
datas1=df_11.sort_values(['url'],ascending=False).groupby(['rest_type'],
                as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False).head(10))['url'].reset_index().rename(columns={'url':'count'})


In [ ]:
datas1

In [ ]:
df_11

In [ ]:
mapbox_access_token="pk.eyJ1Ijoic2hhaHVsZXMiLCJhIjoiY2p4ZTE5NGloMDc2YjNyczBhcDBnZnA5aCJ9.psBECQ2nub0o25PgHcU88w"


## Which are the most popular casual dining restaurant chains

In [ ]:
casual=datas1[datas1['rest_type']=='Casual Dining']
casual

- We can see tht **Empire restaurant,Beijing bites and Mani's dum biriyani** are the most popular casual dining restaurant chains in Bangalore.
- We will inspect them further...

## Where are their outlets located?

In [ ]:
def produce_chains(name):
    data_chain=pd.DataFrame(df[df["name"]==name]['location'].value_counts().reset_index())
    #print(data_chain)
    data_chain.columns=['Name','count']
    #print(data_chain)
    data_chain=data_chain.merge(locations,on="Name",how="left").dropna()
    #print(data_chain)
    data_chain['lan'],data_chain['lon']=zip(*data_chain['geo_loc'].values)
    #print(data_chain)
    return data_chain[['Name','count','lan','lon']]


In [ ]:

def produce_trace(data_chain,name):
        data_chain['text']=data_chain['Name']+'<br>'+data_chain['count'].astype(str)
        trace =  go.Scattermapbox(
           
                lat=data_chain['lan'],
                lon=data_chain['lon'],
                mode='markers',
                marker=go.scattermapbox.Marker(
                    size=data_chain['count']*4
                ),
                text=data_chain['text'],name=name
            )
        
        return trace

In [ ]:

data=[] 
for row in casual['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Casual Dining Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

1. We can see that **Mani's dum biriyani** restaurant has half of their restaurants in central Banglore.
2. ** Empire Restaurant** is present in all over Banglore.
3. **Mani's dum Biriyani** has 12 outlets in ulsoor,which is the most in a single location.

## Top quick bites restaurant chains in Banglore


In [ ]:
quick=datas1[datas1['rest_type']=='Quick Bites']
quick 

- Surprisingly  ** Five star chicken** dominates in quick bites restaurant types overtaking famous **Domino's Pizza and McDonald's**.
- Five Star Chicken is a branch of Charoen Pokphand Group Foods, a Thai multinational conglomerate with over $12 billion business in Agro and Food       Industries. Five Star Chicken specialties in fried chicken.

## Where are their outlets located?

In [ ]:
data=[]  
for row in quick['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Quick Bites Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
#fig.show()
py.iplot(fig, filename='Montreal Mapbox')

## Top Cafes of Banglore?

In [ ]:
cafe=datas1[datas1['rest_type']=='Cafe']
cafe

- No surprises,Cafe coffee day restaurant dominated way over other cafe chains in Banglore.
- Cafe coffee day has 96 outlets in Banglore.
- Café Coffee Day was started as a retail restaurant in 1996. The first CCD outlet was set up on July 11, 1996, at Brigade Road, Bengaluru, Karnataka.


## Where are their outlets located?

In [ ]:
data=[]  
for row in cafe['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Cafe Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

In [ ]:
Delivery=datas1[datas1['rest_type']=='Delivery']
Delivery

## Where are their outlets located?

In [ ]:
data=[]  
for row in Delivery['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Delivery Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

In [ ]:
df_12=zomato_orginal_copy.groupby(['meal_type','name']).agg('count')

In [ ]:
datas2=df_12.sort_values(['url'],ascending=False).groupby(['meal_type'],
                as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False).head(10))['url'].reset_index().rename(columns={'url':'count'})


In [ ]:
datas2

In [ ]:
Buffet=datas2[datas2['meal_type']=='Buffet']
Buffet

In [ ]:
data=[]  
for row in Buffet['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Buffet Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

In [ ]:
Delivery=datas2[datas2['meal_type']=='Delivery']
Delivery

In [ ]:
data=[]  
for row in Delivery['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Delivery Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

In [ ]:
Dine_out=datas2[datas2['meal_type']=='Dine-out']
Dine_out

In [ ]:
data=[]  
for row in Dine_out['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Dine Out Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

In [ ]:
Dessert=datas2[datas2['meal_type']=='Desserts']
Dessert

In [ ]:
data=[]  
for row in Dessert['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Dessert Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

In [ ]:
Cafes=datas2[datas2['meal_type']=='Cafes']
Cafes

In [ ]:
data=[]  
for row in Cafes['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Cafes Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

In [ ]:
Drinks_nightlife=datas2[datas2['meal_type']=='Drinks & nightlife']
Drinks_nightlife

In [ ]:
data=[]  
for row in Drinks_nightlife['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Drinks & nightlife Restaurant chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

In [ ]:
Pubs_and_bars=datas2[datas2['meal_type']=='Pubs and bars']
Pubs_and_bars

In [ ]:
data=[]  
for row in Pubs_and_bars['name']:
    data_chain=produce_chains(row) 
    trace_0=produce_trace(data_chain,row)
    data.append(trace_0)



layout = go.Layout(title="Pubs and bars chains locations around Banglore",
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,style="streets",
        center=dict(
            lat=12.96,
            lon=77.59
        ),
        pitch=0,
        zoom=10
    ),
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Montreal Mapbox')

In [ ]:
from PIL import Image

In [ ]:
# Creating a list with all options available
dishes = list(zomato_orginal_copy['dish_liked'].dropna().astype(str).values)
dishes_word_list = []
for lista in [c.split(',') for c in dishes]:
    for word in lista:
        dishes_word_list.append(word.strip())
        
# Creating a Counter for unique options and generating the wordcloud
dished_wc_dict = Counter(dishes_word_list)

# Reading and preparing a mask for serving as wordcloud background
#food_mask = np.array(Image.open("../input/img-icons/delivery_icon.png"))
#food_mask = food_mask[:, :, -1]
#transf_mask = np.ndarray((food_mask.shape[0], food_mask.shape[1]), np.int32)
#for i in range(len(food_mask)):
    #transf_mask[i] = [255 if px == 0 else 0 for px in food_mask[i]]

# Generating the wordcloud    
wordcloud = WordCloud(width=1000, height=500, collocations=False, random_state=42, colormap='rainbow', 
                      background_color='white').generate_from_frequencies(dished_wc_dict)

# Visualizing the WC created
plt.figure(figsize=(20, 17))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('WordCloud for Dish Liked on Bengaluru Restaurants', size=18, pad=20)
plt.show()

## Wordcloud of dishes liked by cuisines

In [ ]:
zomato_orginal_copy['dish_liked']=zomato_orginal_copy['dish_liked'].apply(lambda x : x.split(',') if type(x)==str else [''])
#x=df.groupby('rest_type',as_index=False)['dish_liked'].agg('sum')
#x['dish_liked']=x['dish_liked'].apply(lambda x : list(filter(lambda a : a!='',x)))

In [ ]:
rest=zomato_orginal_copy['cuisines'].value_counts()
rest

In [ ]:
rest1=zomato_orginal_copy['cuisines'].value_counts()[:15].index
def produce_wordcloud(rest):
    
    plt.figure(figsize=(20,30))
    for i,r in enumerate(rest1):
        plt.subplot(5,3,i+1)
        corpus=zomato_orginal_copy[zomato_orginal_copy['cuisines']==r]['dish_liked'].values.tolist()
        corpus=','.join(x  for list_words in corpus for x in list_words)
        wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,
                      width=1500, height=1500).generate(corpus)
        plt.imshow(wordcloud)
        plt.title(r)
        plt.axis("off")
        

        
        
produce_wordcloud(rest)

In [ ]:
rest=zomato_orginal_copy['rest_type'].value_counts()[:12].index
def produce_wordcloud(rest):
    
    plt.figure(figsize=(20,30))
    for i,r in enumerate(rest):
        plt.subplot(4,3,i+1)
        corpus=zomato_orginal_copy[zomato_orginal_copy['rest_type']==r]['dish_liked'].values.tolist()
        corpus=','.join(x  for list_words in corpus for x in list_words)
        wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,
                      width=1500, height=1500).generate(corpus)
        plt.imshow(wordcloud)
        plt.title(r)
        plt.axis("off")
        

        
        
produce_wordcloud(rest)

In [ ]:
zomato_orginal_copy.columns

In [ ]:
#rest1=zomato_orginal_copy['meal_type'].value_counts().index
#rest1


In [ ]:
rest1=zomato_orginal_copy['meal_type'].value_counts()[:6].index
def produce_wordcloud_cuisines(rest1):
    
    plt.figure(figsize=(30,30))
    for i,r in enumerate(rest1):
        plt.subplot(5,2,i+1)
        corpus=zomato_orginal_copy[zomato_orginal_copy['meal_type']==r]['dish_liked'].values.tolist()
        corpus=','.join(x  for list_words in corpus for x in list_words)
        wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,
                      width=4000, height=2000).generate(corpus)
        plt.imshow(wordcloud)
        plt.title(r)
        plt.axis("off")
produce_wordcloud_cuisines(rest1)

## [Analysing Reviews]()<a id="26"></a><br>

- In this section we will go on to prepare reviews dataframe.
- We will extract reviews and ratings of each restaurant and create a dataframe with it.


In [ ]:
#for name,ratings in tqdm(zip(zomato_orginal_copy['name'],zomato_orginal_copy['reviews_list'])):
    #print(name,ratings)

In [ ]:
all_ratings = []

for name,ratings in tqdm(zip(zomato_orginal_copy['name'],zomato_orginal_copy['reviews_list'])):
    ratings = eval(ratings)
    for score, doc in ratings:
        if score:
            score = score.strip("Rated").strip()
            #print(score)
            doc = doc.strip('RATED').strip()
            #print(doc)
            score = float(score)
            #print(score)
            all_ratings.append([name,score, doc])

In [ ]:
rating_df=pd.DataFrame(all_ratings,columns=['name','rating','review'])
rating_df['review']=rating_df['review'].apply(lambda x : re.sub('[^a-zA-Z0-9\s]',"",x))

In [ ]:
rating_df.to_csv("Ratings.csv")

In [ ]:
rating_df.head()

## [WordCloud of Reviews of restaurant chains]()<a id="27"></a><br>

In [ ]:
rest=df['name'].value_counts()[:9].index
def produce_wordcloud(rest):
    
    plt.figure(figsize=(30,30))
    for i,r in enumerate(rest):
        plt.subplot(3,3,i+1)
        corpus=rating_df[rating_df['name']==r]['review'].values.tolist()
        corpus=' '.join(x  for x in corpus)
        wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=False,
                      width=2000, height=2000).generate(corpus)
        plt.imshow(wordcloud)
        plt.title(r)
        plt.axis("off")
        

        
        
produce_wordcloud(rest)

## [Rating distribution]()<a id="28" ></a><br>

In [ ]:
plt.figure(figsize=(7,6))
rating=rating_df['rating'].value_counts()
sns.barplot(x=rating.index,y=rating)
plt.xlabel("Ratings")
plt.ylabel('count')

## [Topic modelling]()<a id="33" ></a><br>

We will do topic modelling for postive and negative comments seperately to understand the different between the two types.

##  [Topic modeling for positive comments]()<a id="34" ></a><br>

- As the first step we will divide comments as negative and positive on the basis on rating provided.
- Comments with rating below 2.5 is classified as negative and greater tham 2.5 as classified as positive.

In [ ]:
rating_df['sent']=rating_df['rating'].apply(lambda x: 1 if int(x)>2.5 else 0)


Now,
   - we will remove stopwords
   - Lemmatize each word
   - Create corpus
   - Tokenize them

In [ ]:
rating_df

In [ ]:
stops=stopwords.words('english')
#print(stops)
lem=WordNetLemmatizer()
#print(lem)
corpus=' '.join(lem.lemmatize(x) for x in rating_df[rating_df['sent']==1]['review'][:3000] if x not in stops)
#print(corpus)
tokens=word_tokenize(corpus)


> Now we will  use **Termfrequency Inverse doc frequency(Tfidf)** to vectorize the tokens.

In [ ]:

vect=TfidfVectorizer()
vect_fit=vect.fit(tokens)
    

**Latent Dirichlet allocation**

In [ ]:
id_map=dict((v,k) for k,v in vect.vocabulary_.items())
vectorized_data=vect_fit.transform(tokens)
gensim_corpus=gensim.matutils.Sparse2Corpus(vectorized_data,documents_columns=False)
ldamodel = gensim.models.ldamodel.LdaModel(gensim_corpus,id2word=id_map,num_topics=5,random_state=34,passes=25)


## [Visualizing output]()<a id="28" ></a><br>

**Word Counts of Topic Keywords**

When it comes to the keywords in the topics, the importance (weights) of the keywords matters. Along with that, how frequently the words have appeared in the documents is also interesting to look.

Let’s plot the word counts and the weights of each keyword in the same chart.

You want to keep an eye out on the words that occur in multiple topics and the ones whose relative frequency is more than the weight. Often such words turn out to be less important. The chart I’ve drawn below is a result of adding several such words to the stop words list in the beginning and re-running the training process.

In [ ]:
counter=Counter(corpus)

In [ ]:
out=[]
topics=ldamodel.show_topics(formatted=False)
for i,topic in topics:
    for word,weight in topic:
        out.append([word,i,weight,counter[word]])

dataframe = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        


# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(8,6), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=dataframe.loc[dataframe.topic_id==i, :], color=cols[i], width=0.3, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=dataframe.loc[dataframe.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    #ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=8)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(dataframe.loc[dataframe.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=8, y=1.05)    
plt.show()

## [Topic modeling for negative comments]()<a id="35" ></a><br>

In [ ]:
stops=stopwords.words('english')
lem=WordNetLemmatizer()
corpus=' '.join(lem.lemmatize(x) for x in rating_df[rating_df['sent']==0]['review'][:3000] if x not in stops)
tokens=word_tokenize(corpus)

In [ ]:
vect=TfidfVectorizer()
vect_fit=vect.fit(tokens)
id_map=dict((v,k) for k,v in vect.vocabulary_.items())
vectorized_data=vect_fit.transform(tokens)
gensim_corpus=gensim.matutils.Sparse2Corpus(vectorized_data,documents_columns=False)
ldamodel = gensim.models.ldamodel.LdaModel(gensim_corpus,id2word=id_map,num_topics=5,random_state=34,passes=25)

 

In [ ]:
counter=Counter(corpus)
out=[]
topics=ldamodel.show_topics(formatted=False)
for i,topic in topics:
    for word,weight in topic:
        out.append([word,i,weight,counter[word]])

dataframe = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        


# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(8,6), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=dataframe.loc[dataframe.topic_id==i, :], color=cols[i], width=0.3, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=dataframe.loc[dataframe.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    #ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=8)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(dataframe.loc[dataframe.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=8, y=1.05)    
plt.show()

- We can clearly observe the difference between the two types of comments
- The words used are clearly distinguishable.
- The words used in negative comments are clearly critisizing.
- The word used in positive comments  are clearly appreciating.

### T-SNE of Reviews 

(t-SNE) t-Distributed Stochastic Neighbor Embedding is a non-linear dimensionality reduction algorithm used for exploring high-dimensional data. It maps multi-dimensional data to two or more dimensions suitable for human observation. With help of the t-SNE algorithms, you may have to plot fewer exploratory data analysis plots next time you work with high dimensional data.
![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/01/19100947/20131959678_bf1a8e3fcc_b-768x798.jpg)

In this section we will visualize words used in reviews in a 2 dimensional space.
- For that we will first lemmatize and tokenize each reviews and build a corpus out of it.

In [ ]:
stops=set(stopwords.words('english'))
lem=WordNetLemmatizer()
corpus=[]
for review in tqdm(rating_df['review'][:10000]):
    words=[]
    for x in word_tokenize(review):
        x=lem.lemmatize(x.lower())
        if x not in stops:
            words.append(x)
            
    corpus.append(words)

Now we will use word2vec to represent each word as a vector.

In [ ]:
model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=200, workers=4)

In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(10, 10)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()


In [ ]:
tsne_plot(model)

### T-SNE of adjectivs used in postive reviews

In [ ]:
postive=rating_df[rating_df['rating']>3]['review'][:2000]
negative=rating_df[rating_df['rating']<2.5]['review'][:2000]

def return_corpus(df):
    corpus=[]
    for review in df:
        tagged=nltk.pos_tag(word_tokenize(review))
        adj=[]
        for x in tagged:
            if x[1]=='JJ':
                adj.append(x[0])
        corpus.append(adj)
    return corpus

Wow,we can observe all the adjective used in the postive reviews represented in the 2D space.

In [ ]:
corpus=return_corpus(postive)
model = word2vec.Word2Vec(corpus, size=100, min_count=10,window=20, workers=4)
tsne_plot(model)

### T-SNE of adjectives used in Negative reviews

In [ ]:
corpus=return_corpus(negative)
model = word2vec.Word2Vec(corpus, size=100, min_count=10,window=20, workers=4)
tsne_plot(model)

## [Sentimental Analysis]()<a id="29" ></a><br>

Sentiment Analysis is the process of computationally determining whether a piece of writing is positive, negative or neutral. It’s also known as opinion mining, deriving the opinion or attitude of a speaker.



![](https://www.geeksforgeeks.org/wp-content/uploads/Twitter-Sentiment-Analysis-using-Python.jpg)

## [Data preparation]()<a id="30" ></a><br>


For doing sentimental analysis on reviews provided bt users.We have to prepare our data in appropriate format.
We will map reviews to positive and negative on the basis of the ratings provided by each user.So,we will map reviews to negative if the rating given is less than 2.5 and positive if rating is greater than 2.5 


In [ ]:
rating_df['sent']=rating_df['rating'].apply(lambda x: 1 if int(x)>2.5 else 0)

- Next, we will tokenize the data and vectorize the reviews to feed it to our model.


In [ ]:
max_features=3000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(rating_df['review'].values)
X = tokenizer.texts_to_sequences(rating_df['review'].values)
X = pad_sequences(X)


## [Building our model]()<a id="31" ></a><br>

In [ ]:
embed_dim = 32
lstm_out = 32

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

#### Train and test split

- We will now encode our target variable. **pd.get_dummies** is used for on-hot encoding.
- 33 percent of data is reserved for testing our model

In [ ]:
Y = pd.get_dummies(rating_df['sent'].astype(int)).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)



In [ ]:
batch_size = 3200
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size)


### [Validating our model]()<a id="32" ></a><br>

We will take 1500 rows to validate our model.We have choosen **accuacy** to be our evaluation criteria.

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

## Predicting the Success of a Restaurant

Well, finally we arrived at our main task on this notebook: predict the success of a restaurant in Bengaluru using the data provided and extracting some additional features. At this point probably you're asking: but how we will do it? That's what a thought for this task:

1. Use the restaurant rate to classify our data in two classes: good and bad (thresholds to be defined)
2. Create a target variable using a pre-defined rate threshold
3. Extract features from the data
4. Create a classification model using a supervisioned approach
5. Predict the "probability for being a good restaurant" for the ones marked as NEW or without rate


First of all, let's split the data we will use for train/validation and the data we want to do a real prediction.

## **Target Definition**

In [ ]:
cost_dist['rate']

In [ ]:
df['rate_num'] = df['rate'].astype(str).apply(lambda x: x.split('/')[0])
while True:
    try:
        df['rate_num'] = df['rate_num'].astype(float)
        break
    except ValueError as e1:
        noise_entry = str(e1).split(":")[-1].strip().replace("'", "")
        print(f'Threating noisy entrance on rate: {noise_entry}')
        df['rate_num'] = df['rate_num'].apply(lambda x: x.replace(noise_entry, str(np.nan)))

# Dropping old columns

In [ ]:
df['rate_num']

In [ ]:
#df['rate_num']=df['rate_num'].replace('NEW',0.01)

In [ ]:
#df['rate_num'].dropna()

In [ ]:
# Splitting restaur
df['rated'] = df['rate_num'].apply(lambda x: 1 if x >= 0 else 0)
new_restaurants = df.query('rated == 0')
train_val_restaurants = df.query('rated == 1')
#print(train_val_restaurants)
'''my_labels='Rated', 'New or Not Rated'
# PLotting a donut chart for seeing the distribution
#fig, ax = plt.subplots(figsize=(10, 10))
plt.pie(cost_dist[['rated']], labels=my_labels, 
           colors=['darkslateblue', 'silver'])
plt.show()'''
threshold = 3.75
train_val_restaurants['target']=train_val_restaurants['rate_num'].apply(lambda x: 1 if x >= threshold else 0)
#print(train_val_restaurants)

'''# Donut chart
fig, ax = plt.subplots(figsize=(10, 10))
label_names = ['Bad' if target == 0 else 'Good' for target in train_val_restaurants['target'].value_counts().index]
color_list = ['salmon' if label == 'Bad' else 'cadetblue' for label in label_names]
donut_plot(train_val_restaurants, col='target', ax=ax, label_names=label_names, 
           colors=color_list, title='Amount of Good and Bad Restaurants \n(given the selected threshold)',
           text=f'Total of Restaurants:\n{len(train_val_restaurants)}\n\nThreshold: \n{threshold}')'''


#Selecting initial features
initial_features = ['online_order', 'book_table', 'location', 'rest_type', 'cuisines', 
                    'meal_type', 'locality', 'average_cost', 'target']
train_val_restaurants = train_val_restaurants.loc[:, initial_features]
#print(train_val_restaurants)
# Extracting new features
train_val_restaurants['multiple_types'] = train_val_restaurants['rest_type'].astype(str).apply(lambda x: len(x.split(',')))
#print(train_val_restaurants)
train_val_restaurants['total_cuisines'] = train_val_restaurants['cuisines'].astype(str).apply(lambda x: len(x.split(',')))
#print(train_val_restaurants)
# Dropping another ones
#train_val_restaurants.drop('cuisines', axis=1, inplace=True)
train_val_restaurants.head()

In [ ]:
new_restaurants 

Some considerations:

We won't use the votes feature as long as this is a information we only know after launching a restaurant. As we want to be predictive, the idea is to return the probability of success of a restaurant before launching it.
We created the multiple_types, total_dishes and total_cuisines features in a way of counting the food services offered by the restaurant. This is information can be gotten before the launching of the establishment.
For the last act on this topic, let's split the data into training and validation sets once and for all.

In [ ]:
# Splitting the data
X = train_val_restaurants.drop('target', axis=1)
y = train_val_restaurants['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)


## **Encoding**

Looking at the data we prepared for training our classification model, it's important to point the need to apply some encoding technique on categorical features. Here we can define the "global reach" of your restaurant success predictor: using features like location and listed_in(city) for training will restrict the use of our model to Bengaluru area only. Let's keep those ones for awhile pointing that, at least on this first moment, our classifcation model could only be used to predict success of restaurants in Bengaluru.

In [ ]:
# Splitting features by data type
cat_features= [col for col, dtype in X_train.dtypes.items() if dtype == 'object']
num_features = [col for col, dtype in X_train.dtypes.items() if dtype != 'object']

# Apply encoding for categorical features
X_train_cat = X_train[cat_features]
for col in cat_features:
    col_encoded = pd.get_dummies(X_train_cat[col], prefix=col, dummy_na=True)
    X_train_cat = X_train_cat.merge(col_encoded, left_index=True, right_index=True)
    X_train_cat.drop(col, axis=1, inplace=True)
    
print(f'Total categorical features after encoding: {X_train_cat.shape[1]}')

# **Pipeline**

It's probably time to build a complete Pipeline for receiving the raw data with restaurants information and apply all the steps we select for make the data ready for training or prediction. In a first approach, the pipeline will consider:

* Preparing the cost and rate attribute from raw data;
* Selecting initial features to be part of data prep;
* Creating new features based on original data;
* Creating a target for using on training;
* Splitting restaurants based on rated and non-rated ones;
* Encoding the data for categorical features;
* Filling the null data with meadian for numerical features;

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
df.columns

In [ ]:
class PrepareCostAndRate(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Extracting the approx cost feature
        X['average_cost'] = X['approx_cost(for two people)'].astype(str).apply(lambda x: x.replace(',', '.'))
        X['average_cost'] = X['average_cost'].astype(float)
        
        # Extracting the rate feature
        X['rate_num'] = X['rate'].astype(str).apply(lambda x: x.split('/')[0])
        while True:
            try:
                X['rate_num'] = X['rate_num'].astype(float)
                break
            except ValueError as e1:
                noise_entry = str(e1).split(":")[-1].strip().replace("'", "")
                #print(f'Threating noisy entrance on rate feature: {noise_entry}')
                X['rate_num'] = X['rate_num'].apply(lambda x: x.replace(noise_entry, str(np.nan)))              
        
        return X

# Class for selection the initial features
class InitialFeatureSelection(BaseEstimator, TransformerMixin):
    def __init__(self, initial_features=['online_order', 'book_table', 'location', 'rest_type', 'cuisines', 
                                         'listed_in(type)', 'listed_in(city)', 'average_cost', 'rate_num']):
        self.initial_features = initial_features
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.initial_features]
    
# Class for creating some features
class RestaurantAdditionalFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self, multiples_types=True, total_cuisines=True, top_locations=10, top_cities=10, top_types=10):
        self.multiples_types = multiples_types
        self.total_cuisines = total_cuisines
        self.top_locations = top_locations
        self.top_cities = top_cities
        self.top_types = top_types
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        # Adding features based on counting of restaurant types and cuisines
        if self.multiples_types:
            X['multiple_types'] = X['rest_type'].astype(str).apply(lambda x: len(x.split(',')))
        if self.total_cuisines:
            X['total_cuisines'] = X['cuisines'].astype(str).apply(lambda x: len(x.split(',')))
            X.drop('cuisines', axis=1, inplace=True)
            
        # Creating for features for reducing granularity on location
        main_locations = list(X['location'].value_counts().index)[:self.top_locations]
        X['location_feature'] = X['location'].apply(lambda x: x if x in main_locations else 'Other')
        X.drop('location', axis=1, inplace=True)
        
        # Creating for features for reducing granularity on city
        main_cities = (X['listed_in(city)'].value_counts().index)[:self.top_cities]
        X['city_feature'] = X['listed_in(city)'].apply(lambda x: x if x in main_cities else 'Other')
        X.drop('listed_in(city)', axis=1, inplace=True)
        
        # Creating for features for reducing granularity on restaurant type
        main_rest_type = (X['rest_type'].value_counts().index)[:self.top_types]
        X['type_feature'] = X['rest_type'].apply(lambda x: x if x in main_rest_type else
'Other')
        X.drop('rest_type', axis=1, inplace=True)
        
        return X
            
# Class for creating a target based on a threshold (training only)
class CreateTarget(BaseEstimator, TransformerMixin):
    
    def __init__(self, threshold=3.75):
        self.threshold = threshold
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['target'] = X['rate_num'].apply(lambda x: 1 if x >= self.threshold else 0)
        
        return X
    
# Class for splitting the data into new (not rated) and old (rated) restaurants
class SplitRestaurants(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Splits the restaurants based on rate column (rated and non rated)
        rated = X[~X['rate_num'].isnull()]
        non_rated = X[X['rate_num'].isnull()]
        
        # Dropping the rate column
        rated.drop('rate_num', axis=1, inplace=True)
        non_rated.drop('rate_num', axis=1, inplace=True)
        
        return rated, non_rated
                                                 

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Reading raw data
data_path = r'../input/zomato.csv'
raw_data = pd.read_csv(data_path)

# Defining a commoon pipeline to be applied after reading the raw data
common_pipeline = Pipeline([
    ('initial_preparator', PrepareCostAndRate()),
    ('selector', InitialFeatureSelection()),
    ('feature_adder', RestaurantAdditionalFeatures()),
    ('target_creator', CreateTarget()),
    ('new_splitter', SplitRestaurants())
])

# Applying the initial pipeline
train_restaurants, new_restaurants = common_pipeline.fit_transform(raw_data)
print(f'Total restaurants to be used on training: {len(train_restaurants)}')
print(f'Total restaurants to be used on prediction: {len(new_restaurants)}')

Let's take a look at our train_restaurants data:

In [ ]:
train_restaurants.head()

Let's define and apply a categorical and a numerical pipeline for preparing the data:

In [ ]:
import lightgbm as lgb
import shap

In [ ]:
# Splitting into training and testing data
X = train_restaurants.drop('target', axis=1)
y = train_restaurants['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)

# Splitting into cat and num data
cat_features = [col for col, dtype in X_train.dtypes.items() if dtype == 'object']
num_features = [col for col, dtype in X_train.dtypes.items() if dtype != 'object']

# Building a numerical processing pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

# Building a categorical processing pipeline
cat_pipeline = Pipeline([
    ('encoder',pd.get_dummies(dummy_na=True))
])

# Building a complete Pipeline
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

# Applying the full pipeline into the data
X_train_prep = full_pipeline.fit_transform(X_train)
X_test_prep = full_pipeline.fit_transform(X_test)
print(f'Shape of X_train_prep: {X_train_prep.shape}')
print(f'Shape of X_test_prep: {X_test_prep.shape}')

# returning categorical features after encoding and creating a new set of features after the pipeline
encoded_features = full_pipeline.named_transformers_['cat']['encoder'].features_after_encoding
model_features = num_features + encoded_features
print(f'\nSanity check! Number of features after the pipeline (must be the same as shape[1]): {len(model_features)}')

Perfect! We are now ready to train classification models and select the best one for our task!

## **Training a Model**

Finally, we come to the main point of our project: training a classification model to predict the probability of success (high rate) of a given restaurant. We have already went trough about 80% of work done on prep and transformations. So, for making this training step easier, let's use a homemade utility script called ml_utils.py. It contains a class called BinaryClassifierAnalysis with useful methods for training and evaluating selected classifiers.

For this task, let's train a LogisticRegression, DecisionTrees, RandomForest and LightGBM classifiers, each one using RandomizedGridSearchCV with 5 k-folds and hiperparameters pre-defined.

In [ ]:
# Logistic Regression hyperparameters
logreg_param_grid = {
    'C': np.linspace(0.1, 10, 20),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None],
    'random_state': [42],
    'solver': ['liblinear']
}

# Decision Trees hyperparameters
tree_param_grid = {
    'criterion': ['entropy', 'gini'],
    'max_depth': [3, 5, 10, 20],
    'max_features': np.arange(1, X_train.shape[1]),
    'class_weight': ['balanced', None],
    'random_state': [42]
}

# Random Forest hyperparameters
forest_param_grid = {
    'bootstrap': [True, False],
    'max_depth': [3, 5, 10, 20, 50],
    'n_estimators': [50, 100, 200, 500],
    'random_state': [42],
    'max_features': ['auto', 'sqrt'],
    'class_weight': ['balanced', None]
}

# LightGBM hyperparameters
lgbm_param_grid = {
    'num_leaves': list(range(8, 92, 4)),
    'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [3, 4, 5, 6, 8, 12, 16],
    'learning_rate': [0.1, 0.05, 0.01, 0.005],
    'bagging_freq': [3, 4, 5, 6, 7],
    'bagging_fraction': np.linspace(0.6, 0.95, 10),
    'reg_alpha': np.linspace(0.1, 0.95, 10),
    'reg_lambda': np.linspace(0.1, 0.95, 10),
}
lgbm_fixed_params = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

In [ ]:
# Setting up classifiers
set_classifiers = {
    'LogisticRegression': {
        'model': LogisticRegression(),
        'params': logreg_param_grid
    },
    'DecisionTrees': {
        'model': DecisionTreeClassifier(),
        'params': tree_param_grid
    },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': forest_param_grid
    },
    'LightGBM': {
        'model': lgb.LGBMClassifier(**lgbm_fixed_params),
        'params': lgbm_param_grid
    }
}

* Training and evaluating classifiers

In [ ]:
#Creating an instance for the homemade class BinaryClassifiersAnalysis
clf_tool = BinaryClassifiersAnalysis()
clf_tool.fit(set_classifiers, X_train_prep, y_train, random_search=True, cv=5, verbose=5)

In [ ]:
#evaluating metrics
df_performances = clf_tool.evaluate_performance(X_train_prep, y_train, X_test_prep, y_test, cv=5)
df_performances.reset_index(drop=True).style.background_gradient(cmap='Blues')

It's very interesting to see that the RandomForest was the best classification model for our task. Meanwhile it's also the heaviest one in terms of time consuming. Let's use it for further evaluation.

* Plotting the ROC Curve for the classifiers (train and test)

In [ ]:
clf_tool.plot_roc_curve()


* Plotting Confusion Matrix for each classifier (train and test)

In [ ]:
clf_tool.plot_confusion_matrix(classes=['Good', 'Bad'])

* Looking at the feature importance of a specific model

In [ ]:
fig, ax = plt.subplots(figsize=(13, 11))
forest_feature_importance = clf_tool.feature_importance_analysis(model_features, specific_model='LightGBM', ax=ax)
plt.show()

* Looking at score (proba) distribution for a specific model

In [ ]:
clf_tool.plot_score_distribution('LightGBM', shade=True)

In [ ]:
clf_tool.plot_score_bins('LightGBM', bin_range=0.1)

* Using shap analysis for evaluating features patterns

In [ ]:
model = clf_tool.classifiers_info['LightGBM']['estimator']
explainer = shap.TreeExplainer(model)
df_X_train_prep = pd.DataFrame(X_train_prep, columns=model_features)
shap_values = explainer.shap_values(df_X_train_prep)

# Plotting a summary plot using shap
shap.summary_plot(shap_values, df_X_train_prep)

## **Conclusion**

Well, after going trough all the tasks we can conclude that it was possible to predict the success (in terms of rate) for new restaurants put in Zomato service. This is very interesting for business areas to evaluate the main concepts of restaurants and customers preferences in Bengaluru region. With this information, it's clearly possible to take a look at restaurant's features before launching it! The business men and the customers would appreciate it!

So, thinking of a developer way, it would be nice to code this as a production script that receives data from new restaurants monthly (or daily if you wish) and returns the "success score probability". For this to be true, we have to encapsulate a bunch of steps, like building (or reading) a complete pipeline in a pkl format, building (or reading) a classification model also in pkl format and, by the end, applying the predict or predict_proba methods for scoring the new data. Let's simulate it.

In [ ]:
# Applying the full pipeline into new restaurants
new_restaurants_prep = full_pipeline.fit_transform(new_restaurants.drop('target', axis=1))

# Returning the best model and predicting the rate for new restaurants
model = clf_tool.classifiers_info['LightGBM']['estimator']
y_pred = model.predict(new_restaurants_prep)
y_probas = model.predict_proba(new_restaurants_prep)
y_scores = y_probas[:, 1]

# Labelling new data
new_restaurants['success_class'] = y_pred
new_restaurants['success_proba'] = y_scores
new_restaurants.head()

In [ ]:
# Looking at the score distribution for new restaurants
fig, ax = plt.subplots(figsize=(16, 9))
sns.kdeplot(new_restaurants['success_proba'], ax=ax, shade=True, color='mediumseagreen')
format_spines(ax, right_border=False)
ax.set_title('Score Distribution of Success for New Restaurants on the Dataset', size=16, color='dimgrey')
plt.show()


What are the most promissor restaurants (with the highest proba score)?

In [ ]:
# Ordering new restaurants by proba score
new_restaurants_data = new_restaurants.reset_index().merge(raw_data.reset_index()[['name', 'index']], how='left', on='index')
top_new = new_restaurants_data.sort_values(by='success_proba', ascending=False).head(10)
top_new = top_new.loc[:, ['name', 'success_proba', 'online_order', 'book_table', 'listed_in(type)',
                          'approx_cost', 'multiple_types', 'total_cuisines', 'location_feature',
                          'city_feature', 'type_feature']]
top_new

What are the restaurants with the lowest likely probability? (the ones that maybe customers won't order for Zomato)

In [ ]:
# Ordering new restaurants by proba score
bottom_new = new_restaurants_data.sort_values(by='success_proba', ascending=True).head(10)
bottom_new = bottom_new.loc[:, ['name', 'success_proba', 'online_order', 'book_table', 'listed_in(type)',
                          'approx_cost', 'multiple_types', 'total_cuisines', 'location_feature',
                          'city_feature', 'type_feature']]
bottom_new

<font color='red' size=4>  If you like my kernel please consider upvoting.</font>

<font color='green' size=3>Comments are most welcomed !</font>    